In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
from itertools import cycle
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
# hyperparameter tuning ## To search for the most suitable parameters for the Decision Tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from yellowbrick import ROCAUC
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import tree
from IPython.display import Image
from sklearn.metrics import confusion_matrix
import pydotplus
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.xmeans import xmeans, splitting_type
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES

# Data Preparation

In [2]:
X_train = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/y_test.txt", header=None, delim_whitespace=True )
subject_test = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/subject_test.txt", header=None, delim_whitespace=True )
subject_train = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/subject_train.txt", header=None, delim_whitespace=True )
features = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/features.txt", header=None, delim_whitespace=True )

In [3]:
# body_acc_x_train = pd.read_csv("body_acc_x_train.txt", header=None,  delim_whitespace=True)
# body_acc_y_train = pd.read_csv("body_acc_y_train.txt", header=None,  delim_whitespace=True)
# body_acc_z_train = pd.read_csv("body_acc_z_train.txt", header=None,  delim_whitespace=True)
# body_gyro_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
# body_gyro_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
body_gyro_z_train = pd.read_csv("/Users/salvu/Desktop/DM2Git/4-5Modulo/Dataset/body_gyro_z_train.txt", header=None,  delim_whitespace=True)
# total_acc_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
# total_acc_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
# total_acc_z_train = pd.read_csv("body_gyro_z_train.txt", header=None,  delim_whitespace=True)

# body_acc_x_test  = pd.read_csv("body_acc_x_test.txt", header=None,  delim_whitespace=True)
# body_acc_y_test  = pd.read_csv("body_acc_y_test.txt", header=None,  delim_whitespace=True)
# body_acc_z_test  = pd.read_csv("body_acc_z_test.txt", header=None,  delim_whitespace=True)
# body_gyro_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
# body_gyro_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
# body_gyro_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)
# total_acc_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
# total_acc_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
# total_acc_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)

In [4]:
feature = features
feature.drop(0,inplace=True, axis=1)

In [5]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [6]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std

stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)
#for col in features:
#    if (stringa in col):
#        features.drop(labels=col,axis=1,inplace=True)

## Sax_transformation

In [7]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import SymbolicAggregateApproximation

ts=body_gyro_z_train

In [8]:
#normalizz
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series # Normalizza una riga
ts_norm = scaler.fit_transform(ts)
ts_norm.shape

(7352, 128, 1)

In [11]:
n_paa_segments = 24
n_sax_symbols = 6
sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
ts_sax = sax.fit_transform(ts_norm)
ts_sax_inv = sax.inverse_transform(ts_sax)  #Non funziona per incompatibilità delle librerie

In [ ]:
np.unique(ts_sax_inv)    #Per vedere i valori soglia

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(ts_norm[2528], label='Original') #Max var Walking-upstair 2528
plt.plot(ts_sax_inv[2528], label='SAX_Approx') #min var Sitting 3288
#plt.plot(ts_sax[0], label='SAX')
plt.legend(loc='best')
plt.show()

## Sequential Pattern Mining

In [12]:
# !pip install prefixspan

In [13]:
from prefixspan import PrefixSpan

In [14]:
ts_sax=ts_sax.reshape(7352,24)
ps = PrefixSpan(ts_sax)

In [15]:
ts_sax

array([[4, 3, 4, ..., 0, 1, 0],
       [4, 3, 5, ..., 2, 4, 5],
       [2, 1, 2, ..., 4, 3, 1],
       ...,
       [4, 4, 5, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 3, 2],
       [3, 5, 4, ..., 4, 4, 4]])

In [19]:
b=ps.topk(100) #Top k-frequent pattern
b

[(7289, [3]),
 (7255, [2]),
 (7221, [4]),
 (7215, [1]),
 (7206, [0]),
 (7194, [5]),
 (6940, [3, 3]),
 (6877, [2, 2]),
 (6841, [4, 4]),
 (6791, [3, 2]),
 (6753, [4, 3]),
 (6675, [1, 1]),
 (6671, [2, 3]),
 (6664, [0, 0]),
 (6622, [3, 4]),
 (6612, [2, 1]),
 (6548, [1, 2]),
 (6527, [4, 2]),
 (6510, [3, 1]),
 (6493, [5, 5]),
 (6437, [5, 4]),
 (6429, [1, 3]),
 (6406, [1, 0]),
 (6400, [2, 4]),
 (6340, [4, 5]),
 (6339, [0, 1]),
 (6298, [5, 3]),
 (6285, [2, 0]),
 (6262, [4, 1]),
 (6209, [0, 2]),
 (6203, [1, 4]),
 (6180, [3, 3, 3]),
 (6176, [3, 5]),
 (6169, [3, 0]),
 (6116, [0, 3]),
 (6116, [5, 2]),
 (6066, [4, 4, 4]),
 (6042, [2, 2, 2]),
 (6031, [3, 3, 2]),
 (6003, [4, 4, 3]),
 (5988, [3, 2, 2]),
 (5973, [4, 3, 3]),
 (5972, [2, 5]),
 (5953, [4, 0]),
 (5906, [0, 4]),
 (5886, [5, 1]),
 (5871, [2, 3, 3]),
 (5857, [4, 3, 2]),
 (5833, [2, 2, 3]),
 (5823, [3, 3, 4]),
 (5819, [3, 4, 4]),
 (5794, [1, 5]),
 (5763, [3, 2, 3]),
 (5753, [2, 2, 1]),
 (5738, [3, 2, 1]),
 (5709, [4, 4, 2]),
 (5707, [2, 3, 4])

In [20]:
# [( 2, [0]) -> vuol dire che '0' compare 2 volte
# [( 2, [1, 1]) -> vuol dire che '1, 1' compare 2 volte
a= ps.frequent(735)  # param is min freq absolute--> 735 corrisponde al minsupport del 10% (7352/10); 1470 = minsup del 20%

In [21]:
#In questo caso se vogliamo utilizzare "b" che rappresenta la topk
count = 0
newarr = []
for i in range(len(b)):
    if len(b[i][1]) == 5 and count < 6:
        newarr.append(b[i])
        count += 1

In [22]:
#IN questo caso se vogliamo utilizzare "a" che rappresenta i pattern con minsup specificato
#dove prima dobbiamo ordinare i valori per support, in modo tale che i primi che becca per quella dimensione siamo sicuri
#che saranno quelli con support maggiore

result = sorted(a, reverse=True)
result = tuple(result)
print('Sorted Tuple :', result)

count = 0
newarr = []
for i in range(len(result)):
    if len(result[i][1]) == 5 and count < 5:
        newarr.append(result[i])
        count += 1

Sorted Tuple : ((7289, [3]), (7255, [2]), (7221, [4]), (7215, [1]), (7206, [0]), (7194, [5]), (6940, [3, 3]), (6877, [2, 2]), (6841, [4, 4]), (6791, [3, 2]), (6753, [4, 3]), (6675, [1, 1]), (6671, [2, 3]), (6664, [0, 0]), (6622, [3, 4]), (6612, [2, 1]), (6548, [1, 2]), (6527, [4, 2]), (6510, [3, 1]), (6493, [5, 5]), (6437, [5, 4]), (6429, [1, 3]), (6406, [1, 0]), (6400, [2, 4]), (6340, [4, 5]), (6339, [0, 1]), (6298, [5, 3]), (6285, [2, 0]), (6262, [4, 1]), (6209, [0, 2]), (6203, [1, 4]), (6180, [3, 3, 3]), (6176, [3, 5]), (6169, [3, 0]), (6116, [5, 2]), (6116, [0, 3]), (6066, [4, 4, 4]), (6042, [2, 2, 2]), (6031, [3, 3, 2]), (6003, [4, 4, 3]), (5988, [3, 2, 2]), (5973, [4, 3, 3]), (5972, [2, 5]), (5953, [4, 0]), (5906, [0, 4]), (5886, [5, 1]), (5871, [2, 3, 3]), (5857, [4, 3, 2]), (5833, [2, 2, 3]), (5823, [3, 3, 4]), (5819, [3, 4, 4]), (5794, [1, 5]), (5763, [3, 2, 3]), (5753, [2, 2, 1]), (5738, [3, 2, 1]), (5709, [4, 4, 2]), (5707, [2, 3, 4]), (5706, [4, 2, 2]), (5697, [2, 3, 2]), (

In [23]:
result #stampa tutti i risultati

((7289, [3]),
 (7255, [2]),
 (7221, [4]),
 (7215, [1]),
 (7206, [0]),
 (7194, [5]),
 (6940, [3, 3]),
 (6877, [2, 2]),
 (6841, [4, 4]),
 (6791, [3, 2]),
 (6753, [4, 3]),
 (6675, [1, 1]),
 (6671, [2, 3]),
 (6664, [0, 0]),
 (6622, [3, 4]),
 (6612, [2, 1]),
 (6548, [1, 2]),
 (6527, [4, 2]),
 (6510, [3, 1]),
 (6493, [5, 5]),
 (6437, [5, 4]),
 (6429, [1, 3]),
 (6406, [1, 0]),
 (6400, [2, 4]),
 (6340, [4, 5]),
 (6339, [0, 1]),
 (6298, [5, 3]),
 (6285, [2, 0]),
 (6262, [4, 1]),
 (6209, [0, 2]),
 (6203, [1, 4]),
 (6180, [3, 3, 3]),
 (6176, [3, 5]),
 (6169, [3, 0]),
 (6116, [5, 2]),
 (6116, [0, 3]),
 (6066, [4, 4, 4]),
 (6042, [2, 2, 2]),
 (6031, [3, 3, 2]),
 (6003, [4, 4, 3]),
 (5988, [3, 2, 2]),
 (5973, [4, 3, 3]),
 (5972, [2, 5]),
 (5953, [4, 0]),
 (5906, [0, 4]),
 (5886, [5, 1]),
 (5871, [2, 3, 3]),
 (5857, [4, 3, 2]),
 (5833, [2, 2, 3]),
 (5823, [3, 3, 4]),
 (5819, [3, 4, 4]),
 (5794, [1, 5]),
 (5763, [3, 2, 3]),
 (5753, [2, 2, 1]),
 (5738, [3, 2, 1]),
 (5709, [4, 4, 2]),
 (5707, [2, 3, 4])

In [24]:
newarr #stampa i risultati specificando freq e pattern

[(3581, [3, 3, 3, 3, 3]),
 (3509, [4, 4, 3, 3, 3]),
 (3503, [4, 4, 4, 3, 3]),
 (3475, [3, 3, 3, 2, 2]),
 (3468, [3, 3, 2, 2, 2])]

In [25]:
#settiamo il new arr come una stringa che possa essere poi scansionata all interno del dataset 
import re

arrnew=[]
for i in range(len(newarr)):
    print(newarr[i][1])
    string = str(newarr[i][1])
    string = re.sub("\[|\]", "", string)
    string = re.sub("\,", "", string)
    arrnew.append(string)
arrnew[0]    

[3, 3, 3, 3, 3]
[4, 4, 3, 3, 3]
[4, 4, 4, 3, 3]
[3, 3, 3, 2, 2]
[3, 3, 2, 2, 2]


'3 3 3 3 3'

In [27]:
#modifichiamo anche ts_sax in modo tale da poter ricercare lo stesso pattern all'interno di ogni riga
ts_sax_nuovo=[]
for i in range(len(ts_sax)):
    string = str(ts_sax[i])
    string = re.sub("\[|\]", "", string)
    ts_sax_nuovo.append(string)
ts_sax_nuovo

['4 3 4 3 1 2 4 4 2 5 5 1 2 3 3 5 4 3 3 2 1 0 1 0',
 '4 3 5 5 4 4 2 2 1 2 1 1 0 1 1 1 3 0 0 4 1 2 4 5',
 '2 1 2 3 0 0 4 2 2 4 5 5 2 1 5 5 3 2 1 1 0 4 3 1',
 '2 5 5 5 4 2 2 1 4 5 2 2 2 2 2 2 2 3 2 0 1 2 2 1',
 '3 3 3 3 4 3 1 1 3 3 2 1 1 0 1 5 1 2 2 0 1 5 5 5',
 '0 0 4 2 1 2 1 1 3 5 5 4 2 1 0 0 2 2 2 4 4 4 4 4',
 '0 0 0 0 1 1 3 2 3 2 2 2 4 3 5 3 1 4 4 2 4 5 5 1',
 '4 4 4 1 4 4 1 4 5 5 2 4 5 5 4 1 1 1 0 0 0 1 0 1',
 '5 5 4 1 2 1 1 0 1 1 3 2 1 4 4 3 3 1 1 3 4 3 4 2',
 '3 3 3 3 1 1 2 4 3 4 2 2 0 0 0 5 4 4 0 3 1 3 5 2',
 '0 0 4 4 4 0 2 2 1 5 2 3 4 4 1 3 4 4 3 3 3 4 2 2',
 '5 0 1 4 5 4 1 1 4 0 0 4 4 2 3 3 5 3 4 2 2 1 2 1',
 '3 4 4 5 4 5 4 4 3 3 3 2 2 2 2 2 1 2 4 1 0 0 0 1',
 '2 3 2 2 2 5 3 1 0 0 1 1 4 1 3 2 1 4 1 4 5 4 3 2',
 '0 0 0 0 1 2 2 2 1 2 2 2 3 3 3 3 3 4 4 5 5 5 5 5',
 '0 0 0 0 0 1 1 1 1 2 2 2 2 3 3 4 4 5 5 5 5 5 5 5',
 '0 0 0 1 1 2 3 2 2 2 2 2 2 1 1 1 3 4 4 3 5 5 5 5',
 '0 0 0 0 2 1 1 4 3 3 3 4 3 3 4 4 4 3 3 3 3 5 4 3',
 '0 3 4 3 1 2 1 0 5 5 1 4 3 3 4 4 3 3 3 5 1 1 2 2',
 '4 5 5 5 5 

In [28]:
pattern_found = []
index1 = []
index2 = []
index3 = []
index4 = []
index5 = []

for i in range(len(arrnew)):    #ciclo i top 5 pattern di lunghezza desiderata precedentemente
    count = 0
    for j in range(len(ts_sax_nuovo)):  #ciclo all'interno del dataset se trovo il pattern.
        #print(ts_sax_nuovo[j], arrnew[i]
        if arrnew[i] in str(ts_sax_nuovo[j]):   #Di ogni pattern ho la relativa lista d'indice della riga.
            count +=1
            if i == 0:
                index1.append(j)
            if i == 1:
                index2.append(j)
            if i == 2:
                index3.append(j)
            if i == 3:
                index4.append(j)
            if i == 4:
                index5.append(j)

    pattern_found.append(count)

In [29]:
#Numero pattern trovati per ognuno.
pattern_found

[228, 314, 313, 276, 276]

## Dataset con ricorrenze dei pattern trovati

In [30]:
y_train_sbilanciato=y_train

4

In [31]:
#mi creo dei vettori per ogni pattern per vedere a quale classe appartiene[1;6]
#pattern 1
a1 = np.count_nonzero(y_train_sbilanciato[0][index1]==1)
a2 = np.count_nonzero(y_train_sbilanciato[0][index1]==2)
a3 = np.count_nonzero(y_train_sbilanciato[0][index1]==3)
a4 = np.count_nonzero(y_train_sbilanciato[0][index1]==4)
a5 = np.count_nonzero(y_train_sbilanciato[0][index1]==5)
a6 = np.count_nonzero(y_train_sbilanciato[0][index1]==6)

#pattern 2
b1 = np.count_nonzero(y_train_sbilanciato[0][index2]==1)
b2 = np.count_nonzero(y_train_sbilanciato[0][index2]==2)
b3 = np.count_nonzero(y_train_sbilanciato[0][index2]==3)
b4 = np.count_nonzero(y_train_sbilanciato[0][index2]==4)
b5 = np.count_nonzero(y_train_sbilanciato[0][index2]==5)
b6 = np.count_nonzero(y_train_sbilanciato[0][index2]==6)

#pattern 3
c1 = np.count_nonzero(y_train_sbilanciato[0][index3]==1)
c2 = np.count_nonzero(y_train_sbilanciato[0][index3]==2)
c3 = np.count_nonzero(y_train_sbilanciato[0][index3]==3)
c4 = np.count_nonzero(y_train_sbilanciato[0][index3]==4)
c5 = np.count_nonzero(y_train_sbilanciato[0][index3]==5)
c6 = np.count_nonzero(y_train_sbilanciato[0][index3]==6)


#pattern 4
d1 = np.count_nonzero(y_train_sbilanciato[0][index4]==1)
d2 = np.count_nonzero(y_train_sbilanciato[0][index4]==2)
d3 = np.count_nonzero(y_train_sbilanciato[0][index4]==3)
d4 = np.count_nonzero(y_train_sbilanciato[0][index4]==4)
d5 = np.count_nonzero(y_train_sbilanciato[0][index4]==5)
d6 = np.count_nonzero(y_train_sbilanciato[0][index4]==6)


#pattern 5
e1 = np.count_nonzero(y_train_sbilanciato[0][index5]==1)
e2 = np.count_nonzero(y_train_sbilanciato[0][index5]==2)
e3 = np.count_nonzero(y_train_sbilanciato[0][index5]==3)
e4 = np.count_nonzero(y_train_sbilanciato[0][index5]==4)
e5 = np.count_nonzero(y_train_sbilanciato[0][index5]==5)
e6 = np.count_nonzero(y_train_sbilanciato[0][index5]==6)


##pattern 6
#f1 = np.count_nonzero(y_train_sbilanciato[0][index6]==1)
#f2 = np.count_nonzero(y_train_sbilanciato[0][index6]==2)
#f3 = np.count_nonzero(y_train_sbilanciato[0][index6]==3)
#f4 = np.count_nonzero(y_train_sbilanciato[0][index6]==4)
#f5 = np.count_nonzero(y_train_sbilanciato[0][index6]==5)
#f6 = np.count_nonzero(y_train_sbilanciato[0][index6]==6)
#

In [32]:
d= {"P1" : [a1,a2,a3,a4,a5,a6], "P2" : [b1,b2,b3,b4,b5,b6], "P3" : [c1,c2,c3,c4,c5,c6], "P4" : [d1,d2,d3,d4,d5,d6], "P5" : [e1,e2,e3,e4,e5,e6]}
new=pd.DataFrame(data=d)

In [33]:
#DataFrame per cui ho le ricorrenze dei pattern trovati in relazione alle classi.
new

,P1,P2,P3,P4,P5
0,3,8,0,4,5
1,9,32,36,6,4
2,1,5,0,3,4
3,82,87,96,76,75
4,78,47,40,52,46
5,55,135,141,135,142
